
#The following instructions can be used to train a Transformer model on the [IWSLT'14 German to English dataset](http://workshop2014.iwslt.org/downloads/proceeding.pdf).


In [ ]:

%%bash 

cd /mnt/c/translator
cd examples/translation/
bash prepare-wmt14en2de.sh
cd ../..

TEXT=examples/translation/wmt17_en_de
fairseq-preprocess ^
    --source-lang en --target-lang de \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/wmt17_en_de --thresholdtgt 0 --thresholdsrc 0 \
    --workers 20
    
mkdir -p checkpoints/fconv_wmt_en_de
# fairseq-train \
#     data-bin/wmt17_en_de \
#     --arch fconv_wmt_en_de \
#     --dropout 0.2 \
#     --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
#     --optimizer nag --clip-norm 0.1 \
#     --lr 0.5 --lr-scheduler fixed --force-anneal 50 \
#     --max-tokens 4000 \
#     --save-dir checkpoints/fconv_wmt_en_de
# alias python=python3


    
# cd examples/translation/
# bash wmt14en-de.sh
# cd ../..

# # Preprocess/binarize the data
# TEXT=examples/translation/wmt14en2de.tokenized.de-en
# fairseq-preprocess --source-lang de --target-lang en \
#     --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
#     --destdir data-bin/wmt14en2de.tokenized.de-en \
#     --workers 20

: 

In [2]:
# %%bash 
# CUDA_VISIBLE_DEVICES=0 fairseq-train \
#     data-bin/wmt17_en_de \
#     --arch meta_transformer_wmt_de_en --share-decoder-input-output-embed \
#     --user-dir transformer\
#     --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
#     --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
#     --dropout 0.3 --weight-decay 0.0001 \
#     --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
#     --max-tokens 500 \
#     --eval-bleu \
#     --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
#     --eval-bleu-detok moses \
#     --eval-bleu-remove-bpe \
#     --eval-bleu-print-samples \
#     --best-checkpoint-metric bleu --maximize-best-checkpoint-metric
  fairseq-train `
    D:/translator/data-bin/wmt17_de_en_pretrain `
    --user-dir D:/translator/transformer `
    --arch meta_transformer_wmt_en_de_big --share-decoder-input-output-embed `
    --restore-file D:/translator/checkpoints/model.pt `
    --optimizer adam --adam-betas "(0.9,0.98)" --clip-norm 0.0 `
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 `
    --dropout 0.3 --weight-decay 0.0001 `
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 `
    --max-tokens 1500 `
    --eval-bleu `
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' `
    --eval-bleu-detok moses `
    --eval-bleu-remove-bpe `
    --eval-bleu-print-samples `
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric `
    --reset-dataloader `
    --reset-lr-scheduler `
    --reset-meters `
    --reset-optimizer 

    
    


Traceback (most recent call last):
  File "/usr/local/bin/fairseq-train", line 8, in <module>
    sys.exit(cli_main())
  File "/usr/local/lib/python3.10/dist-packages/fairseq_cli/train.py", line 541, in cli_main
    parser = options.get_training_parser()
  File "/usr/local/lib/python3.10/dist-packages/fairseq/options.py", line 38, in get_training_parser
    parser = get_parser("Trainer", default_task)
  File "/usr/local/lib/python3.10/dist-packages/fairseq/options.py", line 234, in get_parser
    utils.import_user_module(usr_args)
  File "/usr/local/lib/python3.10/dist-packages/fairseq/utils.py", line 491, in import_user_module
    importlib.import_module(module_name)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 

CalledProcessError: Command 'b'CUDA_VISIBLE_DEVICES=0 fairseq-train \\\n    data-bin/wmt17_en_de \\\n    --arch meta-transformer_wmt_de_en --share-decoder-input-output-embed \\\n    --user-dir models/transformer_legacy.py\\\n    --optimizer adam --adam-betas \'(0.9, 0.98)\' --clip-norm 0.0 \\\n    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \\\n    --dropout 0.3 --weight-decay 0.0001 \\\n    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \\\n    --max-tokens 500 \\\n    --eval-bleu \\\n    --eval-bleu-args \'{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}\' \\\n    --eval-bleu-detok moses \\\n    --eval-bleu-remove-bpe \\\n    --eval-bleu-print-samples \\\n    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric\n'' returned non-zero exit status 1.

In [ ]:
%%bash
fairseq-generate data-bin/iwslt14.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

In [8]:
import torch

# Load an En-Fr Transformer model trained on WMT'14 data :
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en', tokenizer='moses', bpe='fastbpe')

paraphrase = de2en.translate(en2de.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch Hub is a fantastic interface!'

# Compare the results with English-Russian round-trip translation:
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

paraphrase = ru2en.translate(en2ru.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch is a great interface!'

Using cache found in C:\Users\18816/.cache\torch\hub\pytorch_fairseq_main


KeyboardInterrupt: 